Use LLaMA-Factory to fine-tune already available instruct data use LoRA technique

In [1]:
!nvidia-smi

Sat Aug  3 09:26:31 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P0              42W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
# Install Dependencies

%cd /content/

%rm -rf LLaMA-Factory

!git clone https://github.com/hiyouga/LLaMA-Factory.git

%cd LLaMA-Factory

%ls

!pip install -q -e ".[torch,metrics]"

#!pip install -e .[torch,bitsandbytes]

/content
Cloning into 'LLaMA-Factory'...
remote: Enumerating objects: 15962, done.
remote: Counting objects: 100% (6780/6780), done.
remote: Compressing objects: 100% (276/276), done.
remote: Total 15962 (delta 6517), reused 6506 (delta 6504), pack-reused 9182
Receiving objects: 100% (15962/15962), 222.36 MiB | 20.29 MiB/s, done.
Resolving deltas: 100% (11889/11889), done.
/content/LLaMA-Factory
assets/       docker/      LICENSE      pyproject.toml  requirements.txt  src/
CITATION.cff  evaluation/  Makefile     README.md       scripts/          tests/
data/         examples/    MANIFEST.in  README_zh.md    setup.py
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5

In [3]:
# Show help information.
!llamafactory-cli help

2024-08-03 09:28:09.279752: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-03 09:28:09.297192: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-03 09:28:09.318152: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-03 09:28:09.324589: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-03 09:28:09.339783: I tensorflow/core/platform/cpu_feature_guar

In [4]:
import sys
sys.path.append('/content/LLaMA-Factory/src')

import llamafactory

import json
import yaml

print("import Successful!")

import Successful!


In [44]:
# Check pretrain-data example dataset
example_dataset_path = "/content/LLaMA-Factory/data/glaive_toolcall_en_demo.json" # "/content/LLaMA-Factory/data/alpaca_en_demo.json"

with open(example_dataset_path,"r") as f:
  example_dataset = json.load(f)

print(f"Total samples: {len(example_dataset)}")

print(example_dataset[0])

Total samples: 300
{'conversations': [{'from': 'human', 'value': 'Hi, I have some ingredients and I want to cook something. Can you help me find a recipe?'}, {'from': 'gpt', 'value': 'Of course! I can help you with that. Please tell me what ingredients you have.'}, {'from': 'human', 'value': 'I have chicken, bell peppers, and rice.'}, {'from': 'function_call', 'value': '{"name": "search_recipes", "arguments": {"ingredients": ["chicken", "bell peppers", "rice"]}}'}, {'from': 'observation', 'value': '{"recipes": [{"name": "Chicken and Bell Pepper Stir Fry", "ingredients": ["chicken", "bell peppers", "rice"], "instructions": "Cut the chicken into small pieces. Slice the bell peppers. Cook the rice. Stir fry the chicken and bell peppers. Serve over rice."}, {"name": "Chicken and Rice Casserole", "ingredients": ["chicken", "bell peppers", "rice"], "instructions": "Cook the chicken and rice separately. Mix them together with the bell peppers in a casserole dish. Bake until golden brown."}]}'

In [45]:
%pwd

'/content/LLaMA-Factory'

In [46]:
!mkdir configs

mkdir: cannot create directory ‘configs’: File exists


In [79]:
# Load fine-tune config
example_config_path = "/content/LLaMA-Factory/examples/train_lora/llama3_lora_sft_ds0.yaml" #"/content/LLaMA-Factory/examples/train_lora/llama3_lora_sft.yaml"
custom_config_path = "./configs/custom_lora_instruct.json"

with open(example_config_path,"r") as f:
  args = yaml.safe_load(f)

print(json.dumps(args,indent=2))

{
  "model_name_or_path": "meta-llama/Meta-Llama-3-8B-Instruct",
  "stage": "sft",
  "do_train": true,
  "finetuning_type": "lora",
  "lora_target": "all",
  "deepspeed": "examples/deepspeed/ds_z0_config.json",
  "dataset": "identity,alpaca_en_demo",
  "template": "llama3",
  "cutoff_len": 1024,
  "max_samples": 1000,
  "overwrite_cache": true,
  "preprocessing_num_workers": 16,
  "output_dir": "saves/llama3-8b/lora/sft",
  "logging_steps": 10,
  "save_steps": 500,
  "plot_loss": true,
  "overwrite_output_dir": true,
  "per_device_train_batch_size": 1,
  "gradient_accumulation_steps": 2,
  "learning_rate": 0.0001,
  "num_train_epochs": 3.0,
  "lr_scheduler_type": "cosine",
  "warmup_ratio": 0.1,
  "bf16": true,
  "ddp_timeout": 180000000,
  "val_size": 0.1,
  "per_device_eval_batch_size": 1,
  "eval_strategy": "steps",
  "eval_steps": 500
}


In [80]:
# Custom example config
args["model_name_or_path"] = "unsloth/llama-3-8b-Instruct-bnb-4bit"
args["num_train_epochs"] = 5
args["output_dir"] = "saves/llama-3-8b-Instruct-bnb-4bit/lora/pretrain"
args["per_device_train_batch_size"] = 16 #1,2,4,8,16=1min

# Export your custom config
json.dump(args, open(custom_config_path, "w", encoding="utf-8"), indent=2)

print(json.dumps(args,indent=2))

{
  "model_name_or_path": "unsloth/llama-3-8b-Instruct-bnb-4bit",
  "stage": "sft",
  "do_train": true,
  "finetuning_type": "lora",
  "lora_target": "all",
  "deepspeed": "examples/deepspeed/ds_z0_config.json",
  "dataset": "identity,alpaca_en_demo",
  "template": "llama3",
  "cutoff_len": 1024,
  "max_samples": 1000,
  "overwrite_cache": true,
  "preprocessing_num_workers": 16,
  "output_dir": "saves/llama-3-8b-Instruct-bnb-4bit/lora/pretrain",
  "logging_steps": 10,
  "save_steps": 500,
  "plot_loss": true,
  "overwrite_output_dir": true,
  "per_device_train_batch_size": 16,
  "gradient_accumulation_steps": 2,
  "learning_rate": 0.0001,
  "num_train_epochs": 5,
  "lr_scheduler_type": "cosine",
  "warmup_ratio": 0.1,
  "bf16": true,
  "ddp_timeout": 180000000,
  "val_size": 0.1,
  "per_device_eval_batch_size": 1,
  "eval_strategy": "steps",
  "eval_steps": 500
}


In [84]:
!pip install -q -U bitsandbytes
!pip install -q deepspeed>=0.9.3

In [85]:
!rm -rf /content/LLaMA-Factory/saves

In [87]:
# Start training
!FORCE_TORCHRUN=1 llamafactory-cli train /content/LLaMA-Factory/configs/custom_lora_instruct.json

2024-08-03 10:58:14.626954: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-03 10:58:14.649237: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-03 10:58:14.656075: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-03 10:58:15.933009: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[2024-08-03 10:58:18,835] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: pleas

In [106]:
import sys
sys.path.append('/content/LLaMA-Factory/src')

import llamafactory

from llamafactory.chat import ChatModel
from llamafactory.extras.misc import torch_gc

In [108]:
args = dict(
  model_name_or_path="unsloth/llama-3-8b-Instruct-bnb-4bit", # use bnb-4bit-quantized Llama-3-8B-Instruct model
  adapter_name_or_path="/saves/llama-3-8b-Instruct-bnb-4bit",            # load the saved LoRA adapters
  template="llama3",                     # same to the one in training
  finetuning_type="lora",                  # same to the one in training
  quantization_bit=4,                    # load 4-bit quantized model
)

print(json.dumps(args,indent=2))


chat_model = ChatModel(args)

messages = []
print("Welcome to the CLI application, use `clear` to remove the history, use `exit` to exit the application.")
while True:
  query = input("\nUser: ")
  if query.strip() == "exit":
    break
  if query.strip() == "clear":
    messages = []
    torch_gc()
    print("History has been removed.")
    continue

  messages.append({"role": "user", "content": query})
  print("Assistant: ", end="", flush=True)

  response = ""
  for new_text in chat_model.stream_chat(messages):
    print(new_text, end="", flush=True)
    response += new_text
  print()
  messages.append({"role": "assistant", "content": response})

torch_gc()


[INFO|tokenization_utils_base.py:2161] 2024-08-03 11:24:15,219 >> loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--unsloth--llama-3-8b-Instruct-bnb-4bit/snapshots/3ee63d4b29d5e8dae2b1021856baae7c82eabf59/tokenizer.json
[INFO|tokenization_utils_base.py:2161] 2024-08-03 11:24:15,220 >> loading file added_tokens.json from cache at None
[INFO|tokenization_utils_base.py:2161] 2024-08-03 11:24:15,222 >> loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--unsloth--llama-3-8b-Instruct-bnb-4bit/snapshots/3ee63d4b29d5e8dae2b1021856baae7c82eabf59/special_tokens_map.json
[INFO|tokenization_utils_base.py:2161] 2024-08-03 11:24:15,222 >> loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--unsloth--llama-3-8b-Instruct-bnb-4bit/snapshots/3ee63d4b29d5e8dae2b1021856baae7c82eabf59/tokenizer_config.json


{
  "model_name_or_path": "unsloth/llama-3-8b-Instruct-bnb-4bit",
  "adapter_name_or_path": "/saves/llama-3-8b-Instruct-bnb-4bit",
  "template": "llama3",
  "finetuning_type": "lora",
  "quantization_bit": 4
}


[WARNING|logging.py:313] 2024-08-03 11:24:15,557 >> Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


08/03/2024 11:24:15 - INFO - llamafactory.data.template - Replace eos token: <|eot_id|>


INFO:llamafactory.data.template:Replace eos token: <|eot_id|>
[INFO|configuration_utils.py:733] 2024-08-03 11:24:15,609 >> loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--unsloth--llama-3-8b-Instruct-bnb-4bit/snapshots/3ee63d4b29d5e8dae2b1021856baae7c82eabf59/config.json
[INFO|configuration_utils.py:800] 2024-08-03 11:24:15,610 >> Model config LlamaConfig {
  "_name_or_path": "unsloth/llama-3-8b-Instruct-bnb-4bit",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128009,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 8192,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": false,
    "bn

...

08/03/2024 11:24:15 - INFO - llamafactory.model.model_utils.quantization - Loading ?-bit BITSANDBYTES-quantized model.


INFO:llamafactory.model.model_utils.quantization:Loading ?-bit BITSANDBYTES-quantized model.


08/03/2024 11:24:15 - INFO - llamafactory.model.patcher - Using KV cache for faster generation.


INFO:llamafactory.model.patcher:Using KV cache for faster generation.
[WARNING|quantization_config.py:397] 2024-08-03 11:24:15,620 >> Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


ImportError: Using `bitsandbytes` 4-bit quantization requires the latest version of bitsandbytes: `pip install -U bitsandbytes`